In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,StandardScaler
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data = pd.read_csv("dataset/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
data.shape

(10000, 11)

In [4]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data['Gender']

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    0
9998    1
9999    0
Name: Gender, Length: 10000, dtype: int64

In [5]:
onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()

In [7]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [11]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
data.shape

(10000, 13)

In [13]:
X = data.drop('Exited',axis=1)
y = data['Exited']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [56]:
#define a function to use scikeras to find out the best parameters 

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])
    
    return model

In [57]:
# creating the keras classifier
model = KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [58]:
# define the grid serach param
param_grid = {
    'neurons' : [16,32,64,128],
    'layers' : [1,2,3,4],
    'epochs' : [50,100,120],
}

In [59]:
# perform grid search
grid = GridSearchCV(estimator=model,param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)

In [60]:
grid_result = grid.fit(X_train, y_train)
print(f"Best {grid_result.best_score_} using {grid_result.best_params_}")

Fitting 3 folds for each of 48 candidates, totalling 144 fits


/home/akshay/anaconda3/envs/churn_ann/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/akshay/anaconda3/envs/churn_ann/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/akshay/anaconda3/envs/churn_ann/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/akshay/anaconda3/envs/churn_ann/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` w

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
167/167 [==============================] - 2s 5ms/step - loss: 0.5560 - accuracy: 0.7262
Epoch 2/50
167/167 [==============================] - 2s 4ms/step - loss: 0.8342 - accuracy: 0.4228
Epoch 2/50
167/167 [==============================] - 2s 4ms/step - loss: 0.4863 - accuracy: 0.7935
Epoch 2/50
167/167 [==============================] - 2s 4ms/step - loss: 0.5442 - accuracy: 0.7388
Epoch 2/50
167/167 [==============================] - 2s 4ms/step - loss: 0.5375 - accuracy: 0.7672
Epoch 2/50
Epoch 2/50
167/167 [==============================] - 1s 3ms/step - loss: 0.4431 - accuracy: 0.8095
Epoch 3/50
167/167 [==============================] - 2s 4ms/step - loss: 0.4938 - accuracy: 0.7887
Epoch 2/50
167/167 [==============================] - 1s 4ms/step - loss: 0.4404 - accuracy: 0.8072
Epoch 3/50
167/167 [==============================] - 0s 3ms/step - loss: 0.4581 - accuracy: 0.8031
Epoch 3/50
E

/home/akshay/anaconda3/envs/churn_ann/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


167/167 [==============================] - 0s 3ms/step - loss: 0.3263 - accuracy: 0.8660
Epoch 50/50
167/167 [==============================] - 1s 4ms/step - loss: 0.3210 - accuracy: 0.8689
Epoch 50/50
167/167 [==============================] - 1s 4ms/step - loss: 0.3347 - accuracy: 0.8631
Epoch 50/50
  1/167 [..............................] - ETA: 1:45 - loss: 0.7298 - accuracy: 0.4062

...

167/167 [==============================] - 1s 4ms/step - loss: 0.3387 - accuracy: 0.8537
Epoch 4/120
167/167 [==============================] - 1s 5ms/step - loss: 0.1434 - accuracy: 0.9417
Epoch 95/120
167/167 [==============================] - 1s 4ms/step - loss: 0.1136 - accuracy: 0.9533
Epoch 117/120
167/167 [==============================] - 1s 6ms/step - loss: 0.0626 - accuracy: 0.9764
Epoch 75/120
167/167 [==============================] - 1s 4ms/step - loss: 0.3002 - accuracy: 0.8795
Epoch 11/120
167/167 [==============================] - 1s 4ms/step - loss: 0.1525 - accuracy: 0.9366
Epoch 38/120
167/167 [==============================] - 1s 4ms/step - loss: 0.3306 - accuracy: 0.8571
Epoch 5/120
167/167 [==============================] - 1s 4ms/step - loss: 0.1101 - accuracy: 0.9572
Epoch 118/120
167/167 [==============================] - 1s 5ms/step - loss: 0.1355 - accuracy: 0.9439
Epoch 96/120
167/167 [==============================] - 1s 4ms/step - loss: 0.2947 - accuracy: 

/home/akshay/anaconda3/envs/churn_ann/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


167/167 [==============================] - 1s 5ms/step - loss: 0.2757 - accuracy: 0.8851
Epoch 16/120
167/167 [==============================] - 1s 5ms/step - loss: 0.0589 - accuracy: 0.9809
Epoch 80/120
167/167 [==============================] - 1s 6ms/step - loss: 0.1273 - accuracy: 0.9466
Epoch 43/120
167/167 [==============================] - 1s 6ms/step - loss: 0.3423 - accuracy: 0.8601
Epoch 4/120
167/167 [==============================] - 1s 4ms/step - loss: 0.2716 - accuracy: 0.8823
Epoch 17/120
167/167 [==============================] - 1s 5ms/step - loss: 0.0577 - accuracy: 0.9784
Epoch 81/120
167/167 [==============================] - 1s 5ms/step - loss: 0.1178 - accuracy: 0.9522
Epoch 44/120
167/167 [==============================] - 1s 5ms/step - loss: 0.3382 - accuracy: 0.8594
Epoch 5/120
167/167 [==============================] - 1s 6ms/step - loss: 0.2845 - accuracy: 0.8792
Epoch 11/120
167/167 [==============================] - 1s 6ms/step - loss: 0.1121 - accuracy: 0.

/home/akshay/anaconda3/envs/churn_ann/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/120
250/250 [==============================] - 1s 2ms/step - loss: 0.5134 - accuracy: 0.7534
Epoch 2/120
250/250 [==============================] - 1s 2ms/step - loss: 0.4250 - accuracy: 0.8211
Epoch 3/120
250/250 [==============================] - 1s 2ms/step - loss: 0.4104 - accuracy: 0.8270
Epoch 4/120
250/250 [==============================] - 1s 2ms/step - loss: 0.3963 - accuracy: 0.8367
Epoch 5/120
250/250 [==============================] - 1s 2ms/step - loss: 0.3817 - accuracy: 0.8431
Epoch 6/120
250/250 [==============================] - 1s 2ms/step - loss: 0.3696 - accuracy: 0.8479
Epoch 7/120
250/250 [==============================] - 1s 2ms/step - loss: 0.3606 - accuracy: 0.8525
Epoch 8/120
250/250 [==============================] - 1s 2ms/step - loss: 0.3552 - accuracy: 0.8562
Epoch 9/120
250/250 [==============================] - 1s 2ms/step - loss: 0.3511 - accuracy: 0.8558
Epoch 10/120
250/250 [==============================] - 1s 2ms/step - loss: 0.3480 - accura